In [1]:
import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.auto import tqdm


#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
import torchvision.transforms as T

#sklearn
from sklearn.model_selection import StratifiedKFold
from skimage import io

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.append('./detr_custom/')

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
#################################################################

import matplotlib.pyplot as plt

#Glob
from glob import glob

from typing import Iterable, Sequence, List, Tuple, Dict, Optional
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from generators import BlenderStandardDataset

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [8]:
TORCH_CACHE_DIR = 'torch_cache'
DATASET_DIR = '/mnt/blendervol/objdet_std_data'
SQL_TABLE = 'bboxes_std'
BATCH_SIZE = 4

In [9]:
from importlib import reload
import generators
reload(generators)
from generators import BlenderStandardDataset

datagen = BlenderStandardDataset(DATASET_DIR, SQL_TABLE, BATCH_SIZE, shuffle=False)
X, y = datagen[0]

In [10]:
denominator = 0
mean = np.zeros(3)
std = np.zeros_like(mean)
for i, (imgbatch, __) in enumerate(tqdm(datagen, total=len(datagen))):
    imgbatch = np.array(imgbatch)
    n = len(imgbatch)
    mean = np.average((mean, imgbatch.mean((0,1,2))), 0, weights=(denominator, n))
    
    denominator += n

In [77]:
A = np.array([
    np.full((16,16), 1),
    np.full((16,16), 2),
    np.full((16,16), 3)
])[None,...].repeat(10, 0).transpose((0,2,3,1))

B = np.array([
    np.full((16,16), 2),
    np.full((16,16), 4),
    np.full((16,16), 6)
])[None,...].repeat(10, 0).transpose((0,2,3,1))

C = np.array([
    np.full((16,16), 3),
    np.full((16,16), 6),
    np.full((16,16), 9)
])[None,...].repeat(10, 0).transpose((0,2,3,1))

np.mean([A,B,C], (0,1,2,3))

array([2., 4., 6.])

In [78]:
np.std([np.full(16*16*10, 2),
        np.full(16*16*10, 4),
        np.full(16*16*10, 6)])

1.632993161855452

In [79]:
np.std([A,B,C], (0,1,2,3))

array([0.81649658, 1.63299316, 2.44948974])

In [81]:
# Super memory efficient implementation :^)
mean = np.zeros(3)
denominator = 0
for thing in [A, B, C]:
    mean = np.average((mean, thing.mean((0,1,2))), 0, weights=(denominator, len(thing)))
    denominator += len(thing)

print('Mean:')
print(mean)

Mean:
[2. 4. 6.]


In [89]:
np.sqrt((A - mean)**2)

array([[[[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        ...,

        [[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]],

        [[1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.],
         ...,
         [1., 2., 3.],
         [1., 2., 3.],
         [1., 2., 3.]]],


       [[[1., 2., 3.],
         [1., 2., 3.],
         [1., 